### Importing 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [ ]:
path = tf.keras.utils.get_file('reviews.csv', 
                               'https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P')

139264/127831 [================================] - 0s 0us/step


In [ ]:
dataset = pd.read_csv(path)
dataset.head()

,Unnamed: 0,text,sentiment
0,0,So there is no way for me to plug it in here i...,0
1,1,Good case Excellent value.,1
2,2,Great for the jawbone.,1
3,3,Tied to charger for conversations lasting more...,0
4,4,The mic is great.,1


In [ ]:
import string
import re
import nltk
punch = string.punctuation

In [ ]:
def clean_text(text):
    non_punch = ''
    for i in text:
        if i not in string.punctuation:
            non_punch += i
    split_text = re.split('\W+',non_punch)
    stop = nltk.corpus.stopwords.words('english')
    word_lst = []
    for i in split_text:
        if i not in stop:
            word_lst.append(i)
    wn = nltk.WordNetLemmatizer()
    return [wn.lemmatize(word).lower() for word in word_lst]            


In [ ]:
def clean(text):
  s = ''
  for c in text:
    if c not in punch and c.isdigit() == False:
      s += c
  return s.lower()

In [ ]:
# nltk.download('stopwords')

In [ ]:
clean("AS##%$#2sdsd")

'assdsd'

In [ ]:
dataset["text"] = dataset["text"].apply(lambda row:clean(row))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset["text"], dataset["sentiment"], test_size=0.33, random_state=42)

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type = "post"
padding_type = "post"
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token=oov_tok)

In [ ]:
from pygments import token
tokenizer.fit_on_texts(np.array(X_train))
word_index = tokenizer.word_index

In [ ]:
sequence = tokenizer.texts_to_sequences(np.array(X_train)) 
padded = pad_sequences(sequence,maxlen=max_length,padding=padding_type
                       ,truncating=trunc_type)

In [ ]:
test_seq = tokenizer.texts_to_sequences(np.array(X_test)) 
test_pad = pad_sequences(test_seq,maxlen=max_length,padding=padding_type
                       ,truncating=trunc_type)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           16000     
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 6)                 9606      
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
model.fit(padded,np.array(y_train),
          epochs=num_epochs,
          validation_data=(test_pad,np.array(y_test)))

Epoch 1/10
42/42 [==============================] - 1s 8ms/step - loss: 0.6932 - accuracy: 0.5090 - val_loss: 0.6931 - val_accuracy: 0.4863
Epoch 2/10
42/42 [==============================] - 0s 4ms/step - loss: 0.6923 - accuracy: 0.5307 - val_loss: 0.6927 - val_accuracy: 0.5426
Epoch 3/10
42/42 [==============================] - 0s 3ms/step - loss: 0.6894 - accuracy: 0.5772 - val_loss: 0.6902 - val_accuracy: 0.6565
Epoch 4/10
42/42 [==============================] - 0s 3ms/step - loss: 0.6794 - accuracy: 0.5945 - val_loss: 0.6803 - val_accuracy: 0.6505
Epoch 5/10
42/42 [==============================] - 0s 3ms/step - loss: 0.6436 - accuracy: 0.7069 - val_loss: 0.6524 - val_accuracy: 0.6064
Epoch 6/10
42/42 [==============================] - 0s 4ms/step - loss: 0.5514 - accuracy: 0.8381 - val_loss: 0.5880 - val_accuracy: 0.7538
Epoch 7/10
42/42 [==============================] - 0s 3ms/step - loss: 0.4401 - accuracy: 0.8801 - val_loss: 0.5226 - val_accuracy: 0.7796
Epoch 8/10
42/42 [==

In [ ]:
# import io
# out_v = io.open("vecs.tsv","w",encoding='utf-8')
# out_m = io.open("meta.tsv","w",encoding='utf-8')
# for word_num in range(1,vocab_size):
#   word = reverse_word_index[word_num]
#   embeddings = weights[word_num]
#   out_m.write(word+"\n")
#   out_v.write("\t".join([str(x) for x in embeddings])+"\n")

# out_v.close()
# out_m.close()

In [ ]:
def predict(text):
  prod_seq = tokenizer.texts_to_sequences(np.array([text])) 
  prod_pad = pad_sequences(prod_seq,maxlen=max_length,padding=padding_type
                       ,truncating=trunc_type)  
  pred = round(model.predict(prod_pad)[0][0])
  print(text+" : ","positive" if pred == 1 else "negative","sentence")

In [ ]:
predict("no jokes today just pain")
predict("i want 2000 girlfriend")
predict("hey i want to tell you something")
predict("i love you")
predict("you are arrested for stealing my heart")
predict("oh my god i am in love")

no jokes today just pain :  positive sentence
i want 2000 girlfriend :  positive sentence
hey i want to tell you something :  negative sentence
i love you :  positive sentence
you are arrested for stealing my heart :  positive sentence
oh my god i am in love :  positive sentence
